### 03 캐싱(Caching)

- LangChain은 LLM을 위한 선택적 캐싱 레이어를 제공
- 이는 두가지 이유로 유용
  1. 동일한 완료를 여러 번 요청하는 경우 LLM 공급자에 대한 **API 호출 횟수를 줄여 비용을 절감**
  2. LLM 제공업체에 대한 **API 호출 횟수를 줄여 애플리케이션의 속도를 높일 수** 있다
- **SQLite 캐시**: 데이터 베이스 파일을 활용해 캐시 데이터를 저장하여 프로그램을 종료했다가 다시 시작해도 캐시 정보 유지 가능

In [7]:
from dotenv import load_dotenv
from langchain_teddynote import logging

load_dotenv()

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH04-Models")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH04-Models


In [8]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

# 모델을 생성합니다.
llm = ChatOpenAI(model_name="gpt-3.5-turbo")

# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template("{country} 에 대해서 200자 내외로 요약해줘")

# 체인을 생성합니다.
chain = prompt | llm

In [22]:
# 셀의 코드 실행 시간을 측정해주는 Jupyter Notebook 기능
%%time
response = chain.invoke({"country": "한국"})
print(response.content)

UsageError: Line magic function `%%time` not found.


**InMemoryCache**

- 인메모리 캐시를 사용하여 동일 질문에 대한 답변을 저장하고, 캐시에 저장된 답변을 반환

In [7]:
%%time
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache

# 인메모리 캐시를 사용합니다.
set_llm_cache(InMemoryCache())

# 체인을 실행합니다.
response = chain.invoke({"country": "한국"})
print(response.content)

한국은 동아시아에 위치한 대한민국과 조선민주주의인민공화국으로 이루어져 있다. 대한민국은 수도가 서울이며, 무역과 기술 발전으로 경제가 성장하고 있으며 K-pop, K-drama 등 문화 산업으로도 세계적으로 인기를 끌고 있다. 반면 조선민주주의인민공화국은 수도가 평양이며, 개인의 자유와 인권이 제한되어 있는 공산국가이다. 두 국가는 38도선을 경계로 나누어져 있으며, 북한의 핵무기 문제로 국제 사회의 관심을 받고 있다. 한국은 태권도, 한복, 한식 등 다양한 전통문화가 있으며 아름다운 자연환경과 높은 기술력으로 유명하다.
CPU times: user 20.5 ms, sys: 6.11 ms, total: 26.6 ms
Wall time: 4.21 s


In [8]:
%%time
# 체인을 실행합니다.
response = chain.invoke({"country": "한국"})
print(response.content)

한국은 동아시아에 위치한 대한민국과 조선민주주의인민공화국으로 이루어져 있다. 대한민국은 수도가 서울이며, 무역과 기술 발전으로 경제가 성장하고 있으며 K-pop, K-drama 등 문화 산업으로도 세계적으로 인기를 끌고 있다. 반면 조선민주주의인민공화국은 수도가 평양이며, 개인의 자유와 인권이 제한되어 있는 공산국가이다. 두 국가는 38도선을 경계로 나누어져 있으며, 북한의 핵무기 문제로 국제 사회의 관심을 받고 있다. 한국은 태권도, 한복, 한식 등 다양한 전통문화가 있으며 아름다운 자연환경과 높은 기술력으로 유명하다.
CPU times: user 2.9 ms, sys: 1.65 ms, total: 4.55 ms
Wall time: 3.24 ms


**SQLite Cache**

In [2]:
from langchain_community.cache import SQLiteCache
from langchain_core.globals import set_llm_cache
import os

# 캐시 디렉토리를 생성합니다.
if not os.path.exists("cache"):
    os.makedirs("cache")

# SQLiteCache를 사용합니다.
set_llm_cache(SQLiteCache(database_path="cache/llm_cache.db"))

In [9]:
%%time
# 체인을 실행합니다.
response = chain.invoke({"country": "한국"})
print(response.content)

한국은 동아시아에 위치한 고도로 발전한 개발도상국이다. 서울을 수도로 하고 있으며, 인구는 약 5,100만 명이다. 한국은 세계적으로 유명한 문화, 음식, 엔터테인먼트 산업으로 유명하며 K-pop과 한류 등이 세계적으로 큰 인기를 끌고 있다. 또한 기술 혁신과 경제력, 과학기술 발전에도 주목받고 있다. 그러나 북한과의 관계가 긴장되어 있고, 지정학적인 문제와 자연재해로 인한 문제들도 가지고 있어 국제적 관심을 받고 있다. 현재는 코로나19로 인한 경제적 어려움과 사회적 문제들에 직면해 있으며, 미래에 대한 불확실성이 존재한다.
CPU times: user 12.6 ms, sys: 12.2 ms, total: 24.9 ms
Wall time: 34.4 ms
